In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
dir1 = "base/varroas_crop_20"
dir2 = "base/non_varroas_crop_20"

IMG_WIDTH = 20  # Largeur souhaitée de l'image
IMG_HEIGHT = 20  # Hauteur souhaitée de l'image

X = []
y = []

# Parcourir le premier répertoire
for filename in os.listdir(dir1):
    img_path = os.path.join(dir1, filename)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))  # Redimensionner l'image
    # Effectuez ici d'autres traitements sur l'image si nécessaire
    X.append(img)
    y.append(0)  # 0 pour le premier répertoire

# Parcourir le deuxième répertoire
for filename in os.listdir(dir2):
    img_path = os.path.join(dir2, filename)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))  # Redimensionner l'image
    # Effectuez ici d'autres traitements sur l'image si nécessaire
    X.append(img)
    y.append(1)  # 1 pour le deuxième répertoire

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#image = X_train[2]
# Afficher l'image redimensionnée
#cv2.imshow("Image redimensionnée", image)




In [7]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3) ,input_shape = (20,20,3),activation = 'relu'),  
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3) ,activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=50)


Epoch 1/50
153/153 [==============================] - 4s 21ms/step - loss: 0.9305 - accuracy: 0.7767
Epoch 2/50
153/153 [==============================] - 3s 23ms/step - loss: 0.3903 - accuracy: 0.8337
Epoch 3/50
153/153 [==============================] - 4s 24ms/step - loss: 0.3624 - accuracy: 0.8473
Epoch 4/50
153/153 [==============================] - 4s 24ms/step - loss: 0.3399 - accuracy: 0.8548
Epoch 5/50
153/153 [==============================] - 4s 24ms/step - loss: 0.3343 - accuracy: 0.8582
Epoch 6/50
153/153 [==============================] - 4s 24ms/step - loss: 0.3248 - accuracy: 0.8653
Epoch 7/50
153/153 [==============================] - 4s 24ms/step - loss: 0.2929 - accuracy: 0.8805
Epoch 8/50
153/153 [==============================] - 4s 24ms/step - loss: 0.2758 - accuracy: 0.8899
Epoch 9/50
153/153 [==============================] - 4s 24ms/step - loss: 0.2647 - accuracy: 0.8967
Epoch 10/50
153/153 [==============================] - 4s 24ms/step - loss: 0.2604 - accura

In [8]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

103/103 [==============================] - 0s 3ms/step - loss: 0.1958 - accuracy: 0.9314
Test accuracy: 0.9314146041870117
